In [ ]:
! pip install evaluate

In [ ]:
! pip install rank_bm25

In [ ]:
! pip install beir

In [ ]:
! pip install transformers
! pip install tensorflow

In [ ]:
! pip install bert_score

In [ ]:
! pip install rouge_score

In [ ]:
! pip install -U sentence-transformers

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install transformers datasets torch

In [ ]:
!pip install sacremoses

In [ ]:
#collecting all the questions from the benchmark
import json

with open('data_benchmark.json') as f:
    data_bench = json.load(f)

#print(data_bench.values())

question_bench=[]
for val in data_bench.values():
  question_bench.append(val['medical_question'])





In [ ]:
import pandas as pd
df = pd.read_csv("merged_medical_QA_dataset(2).csv")
documents = df[["question", "answer", "source"]].fillna("")

docs = [
    f"Q: {row['question']}\nA: {row['answer']}\nSource:{row['source']}"
    for _, row in df.iterrows() ]

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
json_biogpt_answers = pd.read_json("biogpt_answer_benchmark.json")
answer_bench_biogpt= list(json_biogpt_answers["answer"])
answer_benchmark=[val['response_a'] for val in data_bench.values()]
print(type(answer_bench_biogpt))
print(type(answer_benchmark))

embedding_model= SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
embeddings_biogpt = embedding_model.encode(answer_bench_biogpt, show_progress_bar=True)
embeddings_benchmark = embedding_model.encode(answer_benchmark, show_progress_bar=True)


In [ ]:
embeddings_biogpt= list(embeddings_biogpt)
embeddings_benchmark= list(embeddings_benchmark)

In [ ]:
from evaluate import load
glue_metric= load("glue","mnli")
results= glue_metric.compute(predictions=embeddings_biogpt, references=embeddings_benchmark)
print(results)

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss #facebook AI similarity Search, search engine for vector similarity

embedding_model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")  # Biomedical embeddings ref ds favoris
# a specific embedding model trained on biomedical dataset

# Convert docs to vectors
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1]) #calcul similarité avec la distance euclidienne
index.add(embeddings)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

query_embedding = embedding_model.encode(question_bench,show_progress_bar=True)
# Search top 10 similar docs
_, indices = index.search(query_embedding, k=10) #it was 3 before


reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

retrieved_docs = []
retrieved_sources = []
retrieved_scores= []

for i, ind_list in enumerate(indices):
    query_text = question_bench[i]

    # Créer les paires (query, doc)
    rerank_input = [(query_text, df.iloc[j]["answer"]) for j in ind_list]

    # Appliquer le reranker
    scores = reranker.predict(rerank_input)

    # Trier les documents par score décroissant
    reranked = sorted(zip(scores, ind_list), reverse=True)[:2]  # top 3

    # Reconstituer le contexte et les sources
    context = "\n".join(
        [f"Q: {df.iloc[j]['question']}\nA: {df.iloc[j]['answer']}" for _, j in reranked]
    )
    sources = list(set([df.iloc[j]["source"] for _, j in reranked]))
    top_scores = [score for score, _ in reranked]
    scaler = MinMaxScaler()
    top_scores_array = np.array(top_scores).reshape(-1, 1)
    normalized_scores = scaler.fit_transform(top_scores_array).flatten()

    retrieved_docs.append(context)
    retrieved_sources.append(sources)
    retrieved_scores.append(normalized_scores)

print(retrieved_docs[4])


In [ ]:
import torch
from transformers import AutoTokenizer, BioGptForCausalLM
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

prompts = [f"{retrieved_docs[i]}\n\nQuestion:{question_bench[i]}\n\nAnswer:"
for i in range(len(question_bench))]
#prompt pr chaque query contenant
#1. l'info pertinente (question et réponse ds le dataset medquad)
#2. la question posé
#Answer: (vide)


tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt-large")
biogpt = BioGptForCausalLM.from_pretrained("microsoft/biogpt-large").to(device)

answers=[]
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(device)
    outputs = biogpt.generate(**inputs, max_new_tokens=128)
    answer= tokenizer.decode(outputs[0], skip_special_tokens=True)#retour en format str
    answers.append(answer)




for i, (q, a) in enumerate(zip(question_bench, answers)):
     sources = ",".join(retrieved_sources[i])
     print(f"\nQuestion: {q}\nAnswer: {a}\nSources: {sources}")
     print("Relevance Scores :")
     for score in retrieved_scores[i]:
         print(f"- {score:.4f}")
     print("=" * 50)

In [ ]:
import json
tout2=[]
for q,r in zip(question_bench,answers):
    data_json_biogpt_RAG = {"question": q, "answer": r}
    tout2.append(data_json_biogpt_RAG)
with open("biogpt_answer_benchmark.json", "w", encoding="utf-8") as f:
           json.dump(tout2, f, ensure_ascii=False, indent=4)

  EVALUATING THE ANSWERS AND RANKING SYSTEM WITH BLEU , ROUGE , BERTSCORE, BEIR

In [ ]:
import json
with open("biogpt_answer_benchmark.json", "r", encoding="utf-8") as f:
    data_json_biogpt_RAG = json.load(f)

answer_biogpt=[elt["answer"] for elt in data_json_biogpt_RAG]
answer_benchmark=[val['response_a'] for val in data_bench.values()]



print(len(answer_biogpt))
print(len(answer_benchmark))


In [ ]:
import evaluate

#ROUGE et BLEU évalue me chevauchement lexicale
rouge= evaluate.load("rouge")

results_rouge= rouge.compute(predictions=answer_biogpt, references=answer_benchmark, use_aggregator=True)
print(results_rouge)

In [ ]:
bleu = evaluate.load("bleu")
results_bleu = bleu.compute(predictions=answer_biogpt, references=answer_benchmark)
print(results_bleu)



In [ ]:
import evaluate
import numpy as np
bertscore = evaluate.load("bertscore")
results = bertscore.compute(
    predictions=answer_biogpt,
    references=answer_benchmark,
    lang="en"
)

f1_mean = np.mean(results["f1"])
precision_mean = np.mean(results["precision"])
recall_mean = np.mean(results["recall"])

print(f1_mean, precision_mean, recall_mean)

In [ ]:
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss #facebook AI similarity Search, search engine for vector similarity
from collections import defaultdict
import numpy as np

embedding_model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")  # Biomedical embeddings ref ds favoris

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


corpus, queries, qrels= GenericDataLoader(data_folder="retrieve_bench").load(split="test")

#prepare corpus and data
corpus_ids = list(corpus.keys())
corpus_texts = [corpus[doc_id]['text'] for doc_id in corpus_ids]
query_ids = list(queries.keys())
query_texts = [queries[query_id] for query_id in query_ids]


####### SPARSE RETRIEVAL (BM25) #######
tokenized_corpus = [doc.split() for doc in corpus_texts]
bm25 = BM25Okapi(tokenized_corpus)

sparse_results = {}
for qid, query in zip(query_ids, query_texts):
    scores = bm25.get_scores(query.split())
    ranked = np.argsort(scores)[::-1][:10]
    sparse_results[qid] = {corpus_ids[i]: float(scores[i]) for i in ranked}

####### DENSE RETRIEVAL (FAISS + BioBERT) #######
corpus_embs = embedding_model.encode(corpus_texts, batch_size=32, convert_to_numpy=True)
query_embs= embedding_model.encode(query_texts, batch_size=32, convert_to_numpy=True)

# Build FAISS index
index = faiss.IndexFlatL2(corpus_embs.shape[1])
index.add(corpus_embs)

# Perform search
k = 10
D, I = index.search(query_embs, k)
dense_results = {}
for i, qid in enumerate(query_ids):
    dense_results[qid] = {corpus_ids[j]: float(-D[i][k]) for k, j in enumerate(I[i])}  # inverser L2 pour score élevé

####### RECIPROCAL RANK FUSION (RRF) #######
def reciprocal_rank_fusion(*rankings, k=60):
    fused = defaultdict(float)
    for ranking in rankings:
        for doc_id, score in ranking.items():
            rank = list(ranking.keys()).index(doc_id) + 1
            fused[doc_id] += 1 / (k + rank)
    return dict(sorted(fused.items(), key=lambda x: x[1], reverse=True))

hybrid_results = {}
for qid in query_ids:
    sparse_ranking = sparse_results.get(qid, {})
    dense_ranking = dense_results.get(qid, {})
    fused = reciprocal_rank_fusion(sparse_ranking, dense_ranking)
    hybrid_results[qid] = dict(list(fused.items())[:10])

print("Reranking ongoing...")
rerank_results={}
for qid in tqdm(query_ids, desc="Reranking..."):
    query = queries[qid]
    doc_ids = list(hybrid_results[qid].keys())
    pairs = [(query, corpus[doc_id]["text"]) for doc_id in doc_ids]

    scores = reranker.predict(pairs)
    reranked = sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)

    rerank_results[qid] = {doc_id: float(score) for doc_id, score in reranked}

#evaluate with BEIR
evaluator = EvaluateRetrieval()
ndcg,_map, recall, precision= evaluator.evaluate(qrels, rerank_results, k_values=[1,3,5,10])
mrr= evaluator.evaluate_custom(qrels, rerank_results,metric="mrr",k_values=[1,3,5,10])
print("Hybrid RRF + Rerank results: \n")
for k in [1, 3, 5, 10]:
    print(f"\nMetrics @ {k}")
    print(f"NDCG@{k}:       {ndcg[f'NDCG@{k}']:.4f}")
    print(f"MRR@{k}:        {mrr[f'MRR@{k}']:.4f}")
    print(f"MAP@{k}:        {_map[f'MAP@{k}']:.4f}")
    print(f"Recall@{k}:     {recall[f'Recall@{k}']:.4f}")
    print(f"Precision@{k}:  {precision[f'P@{k}']:.4f}")





In [ ]:
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
from collections import defaultdict
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter

data_path = "retrieve_bench"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

print(f"Corpus size: {len(corpus)} documents")
print(f"Queries: {len(queries)}")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,      # characters per chunk
    chunk_overlap=100,   # overlap between chunks
    length_function=len
)

chunked_corpus = {}
chunk_id = 0

for doc_id, doc_data in corpus.items():
    text = doc_data["text"]
    chunks = text_splitter.split_text(text)
    for chunk in chunks:
        new_id = f"{doc_id}_chunk{chunk_id}"
        chunked_corpus[new_id] = {"text": chunk, "original_doc": doc_id}
        chunk_id += 1

print(f"Chunked corpus size: {len(chunked_corpus)} passages")

# Prepare IDs and texts for retrieval
corpus_ids = list(chunked_corpus.keys())
corpus_texts = [chunked_corpus[doc_id]['text'] for doc_id in corpus_ids]

query_ids = list(queries.keys())
query_texts = [queries[qid] for qid in query_ids]

# -------------------------
# 3. Sparse retrieval (BM25)
# -------------------------
tokenized_corpus = [doc.split() for doc in corpus_texts]
bm25 = BM25Okapi(tokenized_corpus)

sparse_results = {}
for qid, query in zip(query_ids, query_texts):
    scores = bm25.get_scores(query.split())
    ranked = np.argsort(scores)[::-1][:200]
    sparse_results[qid] = {corpus_ids[i]: float(scores[i]) for i in ranked}

# -------------------------
# 4. Dense retrieval (FAISS + BioBERT)
# -------------------------
embedding_model = SentenceTransformer(
    "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
)
corpus_embs = embedding_model.encode(corpus_texts, batch_size=32, convert_to_numpy=True)
query_embs = embedding_model.encode(query_texts, batch_size=32, convert_to_numpy=True)

# Build FAISS index
index = faiss.IndexFlatL2(corpus_embs.shape[1])
index.add(corpus_embs)

k = 10
D, I = index.search(query_embs, k)
dense_results = {}
for i, qid in enumerate(query_ids):
    dense_results[qid] = {corpus_ids[j]: float(-D[i][rank])
                          for rank, j in enumerate(I[i])}  # negative L2 as score

# -------------------------
# 5. Reciprocal Rank Fusion (RRF)
# -------------------------
def reciprocal_rank_fusion(*rankings, k=60):
    fused = defaultdict(float)
    for ranking in rankings:
        for doc_id, score in ranking.items():
            rank = list(ranking.keys()).index(doc_id) + 1
            fused[doc_id] += 1 / (k + rank)
    return dict(sorted(fused.items(), key=lambda x: x[1], reverse=True))




hybrid_results = {}
for qid in query_ids:
    sparse_ranking = sparse_results.get(qid, {})
    dense_ranking = dense_results.get(qid, {})
    fused = reciprocal_rank_fusion(sparse_ranking, dense_ranking)
    hybrid_results[qid] = dict(list(fused.items())[:10])

# # -------------------------
# # 6. Cross-Encoder Reranking
# # -------------------------
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
rerank_results = {}

print("Reranking ongoing...")
for qid in tqdm(query_ids, desc="Reranking"):
    query = queries[qid]
    doc_ids = list(hybrid_results[qid].keys())
    pairs = [(query, chunked_corpus[doc_id]["text"]) for doc_id in doc_ids]

    scores = reranker.predict(pairs)
    reranked = sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)

    # Map chunk IDs back to original document IDs
    rerank_results[qid] = {}
    for chunk_id, score in reranked:
        orig_doc_id = chunked_corpus[chunk_id]["original_doc"]
        rerank_results[qid][orig_doc_id] = max(
            rerank_results[qid].get(orig_doc_id, 0.0),
            float(score)
        )

# -------------------------
# 7. Evaluate with BEIR
# -------------------------
evaluator = EvaluateRetrieval()
fusion_method="zscore"



ndcg, _map, recall, precision = evaluator.evaluate(qrels, rerank_results, k_values=[1, 3, 5, 10])
mrr = evaluator.evaluate_custom(qrels, rerank_results, metric="mrr", k_values=[1, 3, 5, 10])

print("Hybrid RRF + Rerank + Chunking results: \n")
for k in [1, 3, 5, 10]:
    print(f"\nMetrics @ {k}")
    print(f"NDCG@{k}:       {ndcg[f'NDCG@{k}']:.4f}")
    print(f"MRR@{k}:        {mrr[f'MRR@{k}']:.4f}")
    print(f"MAP@{k}:        {_map[f'MAP@{k}']:.4f}")
    print(f"Recall@{k}:     {recall[f'Recall@{k}']:.4f}")
    print(f"Precision@{k}:  {precision[f'P@{k}']:.4f}")

In [ ]:
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
from collections import defaultdict
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter

data_path = "retrieve_bench"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

print(f"Corpus size: {len(corpus)} documents")
print(f"Queries: {len(queries)}")

embedding_model = SentenceTransformer(
    "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
)

corpus_ids, corpus_texts = list(corpus.keys()), [corpus[cid]["text"] for cid in corpus.keys()]
corpus_embeddings = embedding_model.encode(corpus_texts, show_progress_bar=True, convert_to_numpy=True)

# corpus_embs = embedding_model.encode(corpus_texts, batch_size=32, convert_to_numpy=True)
# query_embs = embedding_model.encode(query_texts, batch_size=32, convert_to_numpy=True)

# Build FAISS index
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

k = 10
query_embeddings = embedding_model.encode(list(queries.values()), show_progress_bar=True, convert_to_numpy=True)
D, I = index.search(query_embeddings, k=10)



reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
print("Rerank ongoing...")

run = {}
for i, (qid, qtext) in enumerate(queries.items()):
    ind_list = I[i]
    pairs = [(qtext, corpus[corpus_ids[j]]["text"]) for j in ind_list]
    scores = reranker.predict(pairs)

    # Associer scores aux doc_ids
    scored_pairs = list(zip(ind_list, scores))
    scored_pairs_sorted = sorted(scored_pairs, key=lambda x: x[1], reverse=True)

    run[qid] = {corpus_ids[j]: float(score) for j, score in scored_pairs_sorted}

# rerank_results = {}

# print("Reranking ongoing...")
# for qid in tqdm(query_ids, desc="Reranking"):
#     query = queries[qid]
#     doc_ids = list(hybrid_results[qid].keys())
#     pairs = [(query, chunked_corpus[doc_id]["text"]) for doc_id in doc_ids]

#     scores = reranker.predict(pairs)
#     reranked = sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)

#     # Map chunk IDs back to original document IDs
#     rerank_results[qid] = {}
#     for chunk_id, score in reranked:
#         orig_doc_id = chunked_corpus[chunk_id]["original_doc"]
#         rerank_results[qid][orig_doc_id] = max(
#             rerank_results[qid].get(orig_doc_id, 0.0),
#             float(score)
        #)

# 7. Evaluate with BEIR
# -------------------------
evaluator = EvaluateRetrieval()
fusion_method="zscore"



ndcg, _map, recall, precision = evaluator.evaluate(qrels, run, k_values=[1, 3, 5, 10])
mrr = evaluator.evaluate_custom(qrels, run, metric="mrr", k_values=[1, 3, 5, 10])

print("Pipeline BioBERT result: \n")
for k in [1, 3, 5, 10]:
    print(f"\nMetrics @ {k}")
    print(f"NDCG@{k}:       {ndcg[f'NDCG@{k}']:.4f}")
    print(f"MRR@{k}:        {mrr[f'MRR@{k}']:.4f}")
    print(f"MAP@{k}:        {_map[f'MAP@{k}']:.4f}")
    print(f"Recall@{k}:     {recall[f'Recall@{k}']:.4f}")
    print(f"Precision@{k}:  {precision[f'P@{k}']:.4f}")